> ### Upcoming Quarter's Production Strategy - Key Focus Areas
> Luke,
> 
> I hope this message finds you well. As we approach the next quarter, I want to discuss our production strategy, which will be critical in maintaining our market position and profitability. We have several challenges and opportunities in the upcoming 13 weeks, revolving around our 8 flagship products.
> 
> Here's what we need to focus on:
> 
> - **Adapting to Weekly Production Fluctuations:** Our production capacity will vary due to factors like maintenance and labor availability. It's imperative we adjust our schedules accordingly.
> 
> - **Demand Dynamics:** We're expecting changes in the demand pattern for our products. I need you to keep a close eye on this and ensure we're always a step ahead.
> 
> - **Cost Management:** With the varying costs of production and inventory holding, we must stay cost-effective without compromising our output quality.
> 
> - **Inventory Levels:** We need to manage our inventory smartly, avoiding excessive stock while also preventing shortages.
> 
> I need you to develop a comprehensive production plan that addresses these points. This plan should not only meet our demand but also optimize our costs. I understand this is a complex balancing act, but your expertise in this area is unparalleled.
> 
> I'm confident in your ability to craft a strategy that will guide us through these challenging but potentially rewarding next few months. Let's ensure we're using our resources efficiently while still meeting market demands.
> 
> Looking forward to your plan and recommendations. You already have them, but just in case, I've attached the necessary forecasts for your reference. Thanks in advance.
> 
> Best,\
> John \
> CEO, ABC Co.

$$\min        \quad  C = \sum^{12}_{w=0} \sum^{8}_{p=1} Q_{wp}P_{wp} + L_{wp}H_{wp}$$

$$
\begin{align*}
\min        \quad   &   \sum^{12}_{w=0} \sum^{8}_{p=1} Q_{wp}P_{wp} + L_{wp}H_{wp}              \\
\text{s.t.} \quad   &   L_{0p} = I_{0p} \quad   &\text{for }w = 0,                                \\
            \quad   &   L_{wp} = L_{w-1,p} + Q_{w-1,p} - D_{w-1,p}\quad &\text{for}              \
                        w\in \{1,...,12\},                                                      \\
            \quad   &   Q_{wp} \ge D_{wp} &\text{for}              \
                        w\in \{0,...,12\},                                                     \\
            \quad   &   \sum^{8}_{p=1} Q_{wp} \le C_{w}&\text{for}              \
                        w\in \{0,...,12\},                                        \\
            \quad   &   Z_{wp} \le L_{wp} &\text{for}              \
                        w\in \{1,...,12\}\quad\text{for } p\in\{1,...,8\}                    \\

\end{align*}
$$

In [8]:
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings("ignore")

demand = pd.read_csv("data/demand.csv", index_col=0)
cap = pd.read_csv("data/cap.csv", index_col=0)
min_inv = pd.read_csv("data/min_inv.csv", index_col=0)
max_inv = pd.read_csv("data/max_inv.csv", index_col=0)
init_inv = pd.read_csv("data/init_inv.csv", index_col=0)
hcr = pd.read_csv("data/hcr.csv", index_col=0)
pc = pd.read_csv("data/pc.csv", index_col=0)

hc = np.round(hcr * pc, 2)

data = [demand, cap, min_inv, max_inv, init_inv, hc, pc]
names = ["demand", "cap", "min_inv", "max_inv", "init_inv", "hc", "pc"]

for i, df in enumerate(data):
    print(names[i])
    display(df.head(2))

demand


,P1,P2,P3,P4,P5,P6,P7,P8
week,,,,,,,,
0,131,94,140,100,103,82,101,132
1,81,109,117,81,143,139,100,112


cap


,production_capacity
week,
0,1314
1,1389


min_inv


...

,initial_inventory
product,
P1,119
P2,111


hc


,P1,P2,P3,P4,P5,P6,P7,P8
week,,,,,,,,
0,2.53,3.92,3.01,3.09,3.60,3.02,2.65,2.48
1,3.01,2.48,2.87,3.60,2.75,2.32,2.81,3.83


pc


,P1,P2,P3,P4,P5,P6,P7,P8
week,,,,,,,,
0,744.84,799.45,717.59,701.81,749.39,717.88,736.65,774.42
1,772.09,730.81,754.25,750.88,763.63,725.05,758.99,797.89


In [9]:
from pulp import *

products = demand.columns.to_list()
weeks = demand.index.to_list()
n_products = len(products)
n_weeks = len(weeks)

model = LpProblem("minimize_costs", LpMinimize)

Q = LpVariable.dicts("Q", ((w, p) for w in weeks for p in products), lowBound=0, cat="Continuous")
L = LpVariable.dicts("L", ((w, p) for w in weeks for p in products), lowBound=0, cat="Continuous")

for w in weeks:
    model += lpSum([Q[w, p] * pc.loc[w, p] + L[w, p] * hc.loc[w, p] for p in products])

for p in products:
    model += L[0, p] == init_inv.loc[p]

for w in weeks[1:]:
    for p in products:
        model += L[w, p] == L[w-1, p] + Q[w-1, p] - demand.loc[w-1, p]

for w in weeks:
    for p in products:
        model += Q[w, p] >= demand.loc[w, p]

for w in weeks:
    model += lpSum([Q[w, p] for p in products]) <= cap.loc[w]

for w in weeks:
    for p in products:
        model += L[w, p] >= min_inv.loc[w, p]

model.solve()

if model.status == 1:
    opt_Q = pd.DataFrame()
    opt_L = pd.DataFrame()
    for (w, p) in Q.keys():
        opt_Q.loc[w, p] = Q[w, p].varValue
        opt_L.loc[w, p] = L[w, p].varValue
else:
    print("Infeasible")